# Data Processing pipeline

- index of sentences
- full english sentence (without preprocessing)
- full german sentence  (without preprocessing)
- english sentnece no stop words, punctuation
- german sentnece no stop words, punctuation
- score
- verbs in english (separated by a space and lemmatized)
- verbs in german (separated by a space and lemmatized)
- adjectives in english (separated by a space and lemmatized)
- adjectives in german (separated by a space and lemmatized)
- common nouns in english (separated by a space and lemmatized)
- common nouns in german (separated by a space and lemmatized)
- Nouns of persons
- Entities or organizations


In [4]:
import pandas as pd
import spacy

In [18]:
# nlp_en = spacy.load("en_core_web_md")
# nlp_ge = spacy.load("de_core_news_md")

import en_core_web_md
import de_core_news_md
nlp_en = en_core_web_md.load()
nlp_ge = de_core_news_md.load()

def spacy_analysis(sentence,nlp):
    doc = nlp(sentence)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
              token.shape_, token.is_alpha, token.is_stop,token.vector.shape)

In [12]:
flatten = lambda l: [item for sublist in l for item in sublist]

# some helper functions
def prepare_data(filename):
    data = [l.strip().split() + ['</s>'] for l in open(filename) if l.strip()]
    corpus = flatten(data)
    vocab = set(corpus)
    return vocab, data

In [13]:
def extract_sentences(filename,lower=False):
    if lower:
        data = [l.lower().strip() for l in open(filename) if l.strip()]
    else:
        data = [l.strip() for l in open(filename) if l.strip()]
    return data

### Load Dataset

In [6]:
# sentences_en = pd.DataFrame(extract_sentences('../data/en_de/train.ende.src'),columns = ['sentences_en'])
# sentences_ge = pd.DataFrame(extract_sentences('../data/en_de/train.ende.mt'),columns = ['sentences_ge'])
# scores = pd.read_csv('../data/en_de/train.ende.scores',header=None)
# scores = scores.rename(columns={0:"scores"})

In [7]:
# dataset = pd.merge(sentences_en,sentences_ge,left_index=True,right_index=True)
# dataset = pd.merge(dataset,scores,left_index=True,right_index=True)

### POS extraction

In [15]:
def extract_pos(pos,sentence,nlp):
    doc = nlp(sentence)
    res = ""
    for token in doc:
        if token.pos_==pos:
            res += token.text + " "
    res = res.strip()
    return res

#### Add verbs

In [16]:
#dataset["german_verbs"] = dataset.sentences_ge.apply(lambda x:extract_pos("VERB",x,nlp_ge))
#dataset["english_verbs"] = dataset.sentences_en.apply(lambda x:extract_pos("VERB",x,nlp_en))

#### Add adjectives

In [17]:
#dataset["german_adjectives"] = dataset.sentences_ge.apply(lambda x:extract_pos("ADJ",x,nlp_ge))
#dataset["english_adjectives"] = dataset.sentences_en.apply(lambda x:extract_pos("ADJ",x,nlp_en))

#### Add adverbs

In [19]:
#dataset["german_adverbs"] = dataset.sentences_ge.apply(lambda x:extract_pos("ADV",x,nlp_ge))
#dataset["english_adverbs"] = dataset.sentences_en.apply(lambda x:extract_pos("ADV",x,nlp_en))

#### Add nouns

In [28]:
#dataset["german_nouns"] = dataset.sentences_ge.apply(lambda x:extract_pos("NOUN",x,nlp_ge))
#dataset["english_nouns"] = dataset.sentences_en.apply(lambda x:extract_pos("NOUN",x,nlp_en))

### Remove stopwords

In [19]:
from spacy.lang.en.stop_words import STOP_WORDS as sw_en
from spacy.lang.de.stop_words import STOP_WORDS as sw_ge

In [64]:
def remove_sw(sentence, nlp):
    sentence = nlp(sentence)
    no_sw_sentence = ""
    
    if nlp==nlp_en:
        stop_words=sw_en
    elif nlp==nlp_ge:
        stop_words=sw_ge
    else:
        return('Use valid language: en or ge')
    for token in sentence:
        if token.text not in stop_words:
            no_sw_sentence += token.text + " "
    
    no_sw_sentence=no_sw_sentence.strip()
    return no_sw_sentence

In [65]:
# dataset['english_no_stop_words'] = dataset['sentences_en'].apply(lambda x: remove_sw(x, nlp_en))
# dataset['german_no_stop_words'] = dataset['sentences_ge'].apply(lambda x: remove_sw(x, nlp_ge))

### Lemmatize

In [66]:
def lemmatizer(sentence, nlp):
    sentence = nlp(sentence)
    lemmatized_sentence = ""
    for token in sentence:
        lemmatized_sentence += token.lemma_ + " "
    
    lemmatized_sentence = lemmatized_sentence.strip()
    return lemmatized_sentence

In [67]:
# dataset['english_lemma'] = dataset['sentences_en'].apply (lambda x: lemmatizer(x, nlp_en))
# dataset['german_lemma'] = dataset['sentences_ge'].apply (lambda x: lemmatizer(x, nlp_ge))

### Sentiment Analysis

#### Sentences

In [41]:
from textblob import TextBlob as textblob_en
from textblob_de import TextBlobDE as textblob_ge

In [42]:
def sentiment(sentence, textblob):
    text = textblob(sentence)
    score = text.sentiment.polarity
    return score

In [43]:
# dataset['english_sentence_sentiment'] = dataset['sentences_en'].apply(lambda x: sentiment(x, textblob_en))
# dataset['german_sentence_sentiment'] = dataset['sentences_ge'].apply(lambda x: sentiment(x, textblob_ge))

In [44]:
def stdz(data):
    data_stdz = (data - data.mean())/data.std()
    return data_stdz

In [46]:
# dataset['std_english_sentence_sentiment']= stdz(dataset['english_sentence_sentiment'])
# dataset['std_german_sentence_sentiment']= stdz(dataset['german_sentence_sentiment'])

#### Per POS

#### Store/load dataset

In [73]:
# dataset.to_pickle('../data/dataset_v1.pickle')
dataset = pd.read_pickle("../data/dataset_v1.pickle")

In [74]:
dataset.head()

,sentences_en,sentences_ge,scores,german_verbs,english_verbs,german_adjectives,english_adjectives,german_adverbs,english_adverbs,german_nouns,english_nouns,english_no_stop_words,german_no_stop_words,english_lemma,german_lemma,english_sentence_sentiment,german_sentence_sentiment,std_english_sentence_sentiment,std_german_sentence_sentiment
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,1.101697,besuchte,visited,,,,,,,José Ortega y Gasset visited Husserl Freiburg ...,1934 besuchte José Ortega y Gasset Husserl Fre...,José Ortega y Gasset visit Husserl at Freiburg...,1934 besuchen José Ortega y Gasset Husserl in ...,0.0,0.0,-0.176727,-0.004161
1,"However, a disappointing ninth in China meant ...",Eine enttäuschende Neunte in China bedeutete j...,-0.516656,bedeutete zurückfiel,meant dropped,enttäuschende sechsten,disappointing sixth,jedoch,However back,Neunte Gesamtwertung Platz,ninth standings,"However , disappointing ninth China meant drop...","Eine enttäuschende Neunte China bedeutete , Ge...","however , a disappointing ninth in China mean ...",Eine enttäuschend Neunte in China bedeuten jed...,-0.3,-1.0,-1.754750,-2.444859
2,"In his diary, Chase wrote that the release of ...","In seinem Tagebuch, Chase schrieb, dass die Ve...",-2.226388,schrieb,wrote,,,,,Tagebuch Chase Veröffentlichung Galle Wermut,diary release gall wormwood,"In diary , Chase wrote release Mason Slidell ""...","In Tagebuch , Chase schrieb , Veröffentlichung...","in -PRON- diary , Chase write that the release...","In mein Tagebuch , Chase schreiben , dass der ...",0.0,0.0,-0.176727,-0.004161
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,-0.827379,montiert genannt,mounted called,Schwere,Heavy,,,Arquebuses Waggons,arquebuses wagons,Heavy arquebuses mounted wagons called arquebu...,Schwere Arquebuses Waggons montiert Arquebus à...,heavy arquebus mount on wagon be call arquebus...,Schwere Arquebuses auf Waggon montieren werden...,-0.2,0.0,-1.228742,-0.004161
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,0.364695,abstirbt fressen,die spawning eat,nordpazifische lokale,local bald,fast ausschließlich,usually almost exclusively,Lachs Laichen Regel Glatzadler Lachskörper,salmon eagles salmon carcasses,"Once North Pacific salmon die spawning , usual...","Sobald nordpazifische Lachs Laichen abstirbt ,...",once North Pacific salmon die off after spawn ...,sobald der nordpazifische Lachs nach der Laich...,0.0,0.0,-0.176727,-0.004161
